Скачиваем файлы (я работаю с датасетом Luxury_Beauty)

In [1]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz

--2021-12-05 09:40:50--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73193215 (70M) [application/octet-stream]
Saving to: ‘Luxury_Beauty.json.gz’

Luxury_Beauty.json. 100%[===================>]  69.80M  68.4MB/s    in 1.0s    

2021-12-05 09:40:51 (68.4 MB/s) - ‘Luxury_Beauty.json.gz’ saved [73193215/73193215]



In [3]:
!gzip -d /content/Luxury_Beauty.json.gz

Импортируем все необходимое

In [5]:
!pip install ufal.udpipe

     |████████████████████████████████| 304 kB 6.9 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626630 sha256=c296708275781585957199096919019086a39a0ceee82c8eda8dc4be26efe68a
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe


In [6]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 13.7 MB/s 


In [85]:
import json
import pandas as pd
import ufal.udpipe
import re
from nltk import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import random
import spacy
from nltk.collocations import *
spacy_nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('punkt')
sw = stopwords.words('english')
random.seed(42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
data = []
for line in open('/content/Luxury_Beauty.json', 'r'):
    data.append(json.loads(line))

In [9]:
random.shuffle(data)

In [10]:
data = data[:10000]

In [12]:
df = pd.DataFrame(data)

In [13]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,2.0,True,"08 18, 2015",A15BBWMRP1XF0,B000BIGOY2,{'Color:': ' Light'},Wendy,The product was average. I am only giving it ...,theBalm TimeBalm Concealer,1439856000,NaN,NaN
1,5.0,True,"03 8, 2015",A14LYQMHYIX6F3,B00983KT42,NaN,BlondeOnTour,"Saw improvement in skin in about 30 days, used...",Five Stars,1425772800,NaN,NaN
2,5.0,True,"11 1, 2013",A2DOPOYFI1EVYO,B0006ODQII,NaN,Auntie,I have been using this product for 20 years. I...,Love it,1383264000,NaN,NaN
3,5.0,True,"09 13, 2017",ATAVB4R6QZPYF,B00UGLH286,NaN,HunterTHL,Great product that requires small amount to se...,Five Stars,1505260800,NaN,NaN
4,5.0,True,"04 3, 2012",A1U3YH99KV7UPU,B000J5A2K4,NaN,Gogolatinboy,I love this deodorant...not only smells really...,a KEEPER,1333411200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5.0,True,"11 23, 2014",A30U5CXFOJ7DQ5,B00CA0BMSW,NaN,Obaida Batal,"fits great , work great","fits great, work great",1416700800,NaN,NaN
9996,5.0,True,"10 17, 2017",A1VOVGNX054PQC,B00172JYKW,NaN,Elizabeth von Eschenbach,This stuff is amazing!! Leaves my skin glowin...,Amazing!!,1508198400,NaN,NaN
9997,5.0,True,"08 18, 2010",A17F9KN0FLRLLJ,B000248HLI,{'Format:': ' Health and Beauty'},Mister Casual,My order was shipped on the 17th and I receive...,Excellent!!,1282089600,NaN,NaN
9998,5.0,True,"12 3, 2012",A1SN1YD8KNILJG,B0016191HU,{'Size:': ' 8.45 fl. oz.'},BAR8888,"I have long, thin hair that is dry from highli...",Great for dry hair!,1354492800,NaN,NaN


### 3 способа найти упоминания товаров в отзывах:
* можно было бы написать парсер, который берет слово/нграмму и идет в викидату посмотреть, не относится ли это слово/нграмма к нужной нам группе слов (можно было бы взять "personal care products": Q3246832, и "cosmetics": Q131207)
* можно составить список глаголов, таких как ["buy", "use", "love"]. Затем распарсить предложения с помощью какого-нибудь UDPipe, и для форм выбранных нами глаголов извлекать их прямые объекты
* можно извлечь все конструкции вида ADJ+NP, прогнать их через какой-нибудь классификатор полярности и сохранить те именные группы, которые употребляются с положительными или негативными прилагательными


Я попробую реализовать второй вариант.

In [14]:
# предлагаю в данном дз поработать со следующими глаголами:
verbs = ["buy", "use", "love", "try", "like", "discover", "need", "purchase", "hate"] # для этих глаголов можно брать прямые объекты
verbs_2 = ["work", "break"] # для этих глаголов можно брать субъекты

In [15]:
review_texts = list(df.reviewText)

In [16]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/english-gum-ud-2.5-191206.udpipe?sequence=31&isAllowed=y

--2021-12-05 09:47:06--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/english-gum-ud-2.5-191206.udpipe?sequence=31
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8880020 (8.5M) [application/octet-stream]
Saving to: ‘english-gum-ud-2.5-191206.udpipe?sequence=31’

english-gum-ud-2.5- 100%[===================>]   8.47M  6.47MB/s    in 1.3s    

2021-12-05 09:47:09 (6.47 MB/s) - ‘english-gum-ud-2.5-191206.udpipe?sequence=31’ saved [8880020/8880020]



In [17]:
model = ufal.udpipe.Model.load('/content/english-gum-ud-2.5-191206.udpipe?sequence=31')
tokenizer = model.newTokenizer(model.DEFAULT)
conlluOutput = ufal.udpipe.OutputFormat.newOutputFormat("conllu")
sentence = ufal.udpipe.Sentence()
error = ufal.udpipe.ProcessingError()

In [18]:
def tag(text, model):
    tokenizer.setText(text)
    tokenizer.nextSentence(sentence, error)
    model.tag(sentence, model.DEFAULT)
    model.parse(sentence, model.DEFAULT)
    return conlluOutput.writeSentence(sentence)

In [19]:
reviews_parsed = []
review_id_for_sentence = []
for text in tqdm(review_texts):
  rev_parsed = []
  index_ex = review_texts.index(text)
  if type(text) == str:
    slist = sent_tokenize(text)
    for s in slist:
        s = tag(s, model)
        rev_parsed.append(s)
        review_id_for_sentence.append(index_ex)

  reviews_parsed.append(rev_parsed)

100%|██████████| 10000/10000 [06:42<00:00, 24.85it/s]


In [21]:
print(reviews_parsed[0][0])

# newdoc
# newpar
# sent_id = 1
# text = The product was average.
1	The	the	DET	DT	Definite=Def|PronType=Art	2	det	_	_
2	product	product	NOUN	NN	Number=Sing	4	nsubj	_	_
3	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	4	cop	_	_
4	average	average	ADJ	JJ	Degree=Pos	0	root	_	SpaceAfter=No
5	.	.	PUNCT	.	_	4	punct	_	SpaceAfter=No




In [22]:
df['reviews_ud'] = reviews_parsed

In [26]:
verbs_all = verbs + verbs_2
sentences_all = []
lemmatized_sentences_all = []
sent_targets_all = []

for review in tqdm(reviews_parsed):
  sentences = []
  lemmatized_sentences = []
  sent_targets = []
  sent_lemm_targets = []
  for text in review:
      target = ''
      lemmas = []
      sent = re.findall(r'\n# text = (.*?)\n1\t', text)
      sent = ' '.join(sent)
      sentences.append(sent)
      sent = re.sub(r'[^\w^\s\']', '', sent)
      sent = sent.lower()
      doc = spacy_nlp(sent)
      lemmatized_sent = [token.lemma_ for token in doc]
      for token in lemmatized_sent:
          if token in verbs_all:
              target = token
              
      sent_targets.append(target)   
      lemmatized_sentences.append(' '.join(lemmatized_sent))
  sentences_all.append(sentences)
  lemmatized_sentences_all.append(lemmatized_sentences)
  sent_targets_all.append(sent_targets)

100%|██████████| 10000/10000 [05:25<00:00, 30.76it/s]


In [27]:
df['lemmatized_sentences'] = lemmatized_sentences_all
df['targets'] = sent_targets_all

In [28]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,reviews_ud,lemmatized_sentences,targets
0,2.0,True,"08 18, 2015",A15BBWMRP1XF0,B000BIGOY2,{'Color:': ' Light'},Wendy,The product was average. I am only giving it ...,theBalm TimeBalm Concealer,1439856000,NaN,NaN,[# newdoc\n# newpar\n# sent_id = 1\n# text = T...,"[the product be average, i be only give -PRON-...","[, , use, , purchase]"
1,5.0,True,"03 8, 2015",A14LYQMHYIX6F3,B00983KT42,NaN,BlondeOnTour,"Saw improvement in skin in about 30 days, used...",Five Stars,1425772800,NaN,NaN,[# sent_id = 6\n# text = Saw improvement in sk...,[see improvement in skin in about 30 day use e...,[use]
2,5.0,True,"11 1, 2013",A2DOPOYFI1EVYO,B0006ODQII,NaN,Auntie,I have been using this product for 20 years. I...,Love it,1383264000,NaN,NaN,[# sent_id = 7\n# text = I have been using thi...,"[i have be use this product for 20 year, i be ...","[use, ]"
3,5.0,True,"09 13, 2017",ATAVB4R6QZPYF,B00UGLH286,NaN,HunterTHL,Great product that requires small amount to se...,Five Stars,1505260800,NaN,NaN,[# sent_id = 9\n# text = Great product that re...,[great product that require small amount to se...,"[, ]"
4,5.0,True,"04 3, 2012",A1U3YH99KV7UPU,B000J5A2K4,NaN,Gogolatinboy,I love this deodorant...not only smells really...,a KEEPER,1333411200,NaN,NaN,[# sent_id = 11\n# text = I love this deodoran...,[i love this deodorantnot only smell really go...,"[love, use, ]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5.0,True,"11 23, 2014",A30U5CXFOJ7DQ5,B00CA0BMSW,NaN,Obaida Batal,"fits great , work great","fits great, work great",1416700800,NaN,NaN,"[# sent_id = 33890\n# text = fits great , work...",[fit great work great],[work]
9996,5.0,True,"10 17, 2017",A1VOVGNX054PQC,B00172JYKW,NaN,Elizabeth von Eschenbach,This stuff is amazing!! Leaves my skin glowin...,Amazing!!,1508198400,NaN,NaN,[# sent_id = 33891\n# text = This stuff is ama...,"[this stuff be amazing, leave -PRON- skin glow...","[, , ]"
9997,5.0,True,"08 18, 2010",A17F9KN0FLRLLJ,B000248HLI,{'Format:': ' Health and Beauty'},Mister Casual,My order was shipped on the 17th and I receive...,Excellent!!,1282089600,NaN,NaN,[# sent_id = 33894\n# text = My order was ship...,[-PRON- order be ship on the 17th and i receiv...,"[, love, , ]"
9998,5.0,True,"12 3, 2012",A1SN1YD8KNILJG,B0016191HU,{'Size:': ' 8.45 fl. oz.'},BAR8888,"I have long, thin hair that is dry from highli...",Great for dry hair!,1354492800,NaN,NaN,"[# sent_id = 33898\n# text = I have long, thin...",[i have long thin hair that be dry from highli...,"[, , ]"


In [56]:
def find_arguments_obj(target_id, text):
    list_arguments = []
    list_syntax_roles = []
    list_pos = []
    list_morph_features = []
    match_word = re.findall(r'\n\d*?\t(.*?)\t.*?\t.*?\t.*?\t.*?\t' + target_id + '\t.*?\t_\t.*?\n', text)
    match_syntax = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t.*?\t.*?\t' + target_id + '\t(.*?)\t_\t.*?\n', text)
    match_pos = re.findall(r'\n\d*?\t.*?\t.*?\t(.*?)\t.*?\t.*?\t' + target_id + '\t.*?\t_\t.*?\n', text)
    match_morph = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t.*?\t(.*?)\t' + target_id + '\t.*?\t_\t.*?\n', text)
    for i in range(len(match_word)):
        if (match_syntax[i] == 'obj') and (match_pos[i] != 'PRON'):
            list_arguments.append(match_word[i])
            list_syntax_roles.append(match_syntax[i])
            list_pos.append(match_pos[i])
            list_morph_features.append(match_morph[i])
    
    return list_arguments, list_syntax_roles, list_pos, list_morph_features


def find_arguments_subj(target_id, text):
    list_arguments = []
    list_syntax_roles = []
    list_pos = []
    list_morph_features = []
    match_word = re.findall(r'\n\d*?\t(.*?)\t.*?\t.*?\t.*?\t.*?\t' + target_id + '\t.*?\t_\t.*?\n', text)
    match_syntax = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t.*?\t.*?\t' + target_id + '\t(.*?)\t_\t.*?\n', text)
    match_pos = re.findall(r'\n\d*?\t.*?\t.*?\t(.*?)\t.*?\t.*?\t' + target_id + '\t.*?\t_\t.*?\n', text)
    match_morph = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t.*?\t(.*?)\t' + target_id + '\t.*?\t_\t.*?\n', text)
    for i in range(len(match_word)):
        if (match_syntax[i] == 'nsubj') and (match_pos[i] != 'PRON'):
            list_arguments.append(match_word[i])
            list_syntax_roles.append(match_syntax[i])
            list_pos.append(match_pos[i])
            list_morph_features.append(match_morph[i])

    return list_arguments, list_syntax_roles, list_pos, list_morph_features

In [57]:
def lemmatize(text): 
    doc = spacy_nlp(text)
    return [token.lemma_ for token in doc][0]

In [58]:
targets = list(df.targets)
parsed_sentences = list(df.reviews_ud)
list_all_arguments = []
list_all_syntax_roles = []
list_all_pos = []
list_all_morph_features = []
for i in tqdm(range(len(targets))):
  args = []
  syn = []
  pos = []
  morph = []
  for y in range(len(targets[i])):
    if targets[i][y] != '':
      text_lower = parsed_sentences[i][y].lower()
      match = re.findall(r'\n(\d*?)\t.*?\t' + targets[i][y], text_lower)
      if match:
        target_id = match[0]
        if (lemmatize(targets[i][y].lower()) in verbs):
          list_arguments, list_syntax_roles, list_pos, list_morph_features = find_arguments_obj(target_id, parsed_sentences[i][y])
        elif (lemmatize(targets[i][y].lower()) in verbs_2):
          list_arguments, list_syntax_roles, list_pos, list_morph_features = find_arguments_subj(target_id, parsed_sentences[i][y])

      args = args + list_arguments
      syn = syn + list_syntax_roles
      pos = pos + list_pos
      morph = morph + list_morph_features

  list_all_arguments.append(args)
  list_all_syntax_roles.append(syn)
  list_all_pos.append(pos)
  list_all_morph_features.append(morph)


df['NE'] = list_all_arguments
df['synt_roles'] = list_all_syntax_roles
df['POS'] = list_all_pos
df['morph_features'] = list_all_morph_features

100%|██████████| 10000/10000 [02:12<00:00, 75.21it/s]


In [70]:
 df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,reviews_ud,lemmatized_sentences,targets,NE,synt_roles,POS,morph_features
0,2.0,True,"08 18, 2015",A15BBWMRP1XF0,B000BIGOY2,{'Color:': ' Light'},Wendy,The product was average. I am only giving it ...,theBalm TimeBalm Concealer,1439856000,NaN,NaN,[# newdoc\n# newpar\n# sent_id = 1\n# text = T...,"[the product be average, i be only give -PRON-...","[, , use, , purchase]",[],[],[],[]
1,5.0,True,"03 8, 2015",A14LYQMHYIX6F3,B00983KT42,NaN,BlondeOnTour,"Saw improvement in skin in about 30 days, used...",Five Stars,1425772800,NaN,NaN,[# sent_id = 6\n# text = Saw improvement in sk...,[see improvement in skin in about 30 day use e...,[use],[],[],[],[]
2,5.0,True,"11 1, 2013",A2DOPOYFI1EVYO,B0006ODQII,NaN,Auntie,I have been using this product for 20 years. I...,Love it,1383264000,NaN,NaN,[# sent_id = 7\n# text = I have been using thi...,"[i have be use this product for 20 year, i be ...","[use, ]",[product],[obj],[NOUN],[Number=Sing]
3,5.0,True,"09 13, 2017",ATAVB4R6QZPYF,B00UGLH286,NaN,HunterTHL,Great product that requires small amount to se...,Five Stars,1505260800,NaN,NaN,[# sent_id = 9\n# text = Great product that re...,[great product that require small amount to se...,"[, ]",[],[],[],[]
4,5.0,True,"04 3, 2012",A1U3YH99KV7UPU,B000J5A2K4,NaN,Gogolatinboy,I love this deodorant...not only smells really...,a KEEPER,1333411200,NaN,NaN,[# sent_id = 11\n# text = I love this deodoran...,[i love this deodorantnot only smell really go...,"[love, use, ]",[deodorant...not],[obj],[NOUN],[Number=Sing]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5.0,True,"11 23, 2014",A30U5CXFOJ7DQ5,B00CA0BMSW,NaN,Obaida Batal,"fits great , work great","fits great, work great",1416700800,NaN,NaN,"[# sent_id = 33890\n# text = fits great , work...",[fit great work great],[work],[],[],[],[]
9996,5.0,True,"10 17, 2017",A1VOVGNX054PQC,B00172JYKW,NaN,Elizabeth von Eschenbach,This stuff is amazing!! Leaves my skin glowin...,Amazing!!,1508198400,NaN,NaN,[# sent_id = 33891\n# text = This stuff is ama...,"[this stuff be amazing, leave -PRON- skin glow...","[, , ]",[],[],[],[]
9997,5.0,True,"08 18, 2010",A17F9KN0FLRLLJ,B000248HLI,{'Format:': ' Health and Beauty'},Mister Casual,My order was shipped on the 17th and I receive...,Excellent!!,1282089600,NaN,NaN,[# sent_id = 33894\n# text = My order was ship...,[-PRON- order be ship on the 17th and i receiv...,"[, love, , ]",[],[],[],[]
9998,5.0,True,"12 3, 2012",A1SN1YD8KNILJG,B0016191HU,{'Size:': ' 8.45 fl. oz.'},BAR8888,"I have long, thin hair that is dry from highli...",Great for dry hair!,1354492800,NaN,NaN,"[# sent_id = 33898\n# text = I have long, thin...",[i have long thin hair that be dry from highli...,"[, , ]",[],[],[],[]


### n-граммы с полученными сущностями

In [106]:
ngrams_all = []
nes_all = []
bigrams_per_text = []
for index, row in tqdm(df.iterrows()):
  bigrams = []
  if row['NE'] != []:
    bigrm = list(nltk.bigrams(row['reviewText'].split()))
    for bi in bigrm:
      for ne in row['NE']:
        if ne in bi:
          nes_all.append(ne)
          ngrams_all.append(bi)
          bigrams.append(bi[0] + ' ' + bi[1])
  
  bigrams_per_text.append(bigrams)

df['bigrams'] = bigrams_per_text

10000it [00:02, 4742.55it/s]


### PMI

In [111]:
texts = list(df.reviewText)
texts = [str(text) for text in texts]
texts_all = ' '.join(texts)

In [148]:
best_pmi = []
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(texts_all.split())
finder.apply_freq_filter(5)
best_bgrams = finder.nbest(bigram_measures.pmi, 5000)
for bgram in best_bgrams:
  if bgram in ngrams_all:
    best_pmi.append(bgram)

best_pmi[:10]

[('Seche', 'Vite'),
 ('Pur', 'Minerals'),
 ('Top', 'Coat'),
 ('Calvin', 'Klein'),
 ('Nano', 'Titanium'),
 ('La', 'Roche-Posay'),
 ('Butter', 'London'),
 ('Hot', 'Tools'),
 ('Jane', 'Iredale'),
 ('Jack', 'Black')]

### likelihood ratio

In [150]:
best_likelihood = []
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(texts_all.split())
finder.apply_freq_filter(5)
best_bgrams = finder.nbest(bigram_measures.likelihood_ratio, 5000)
for bgram in best_bgrams:
  if bgram in ngrams_all:
    best_likelihood.append(bgram)

best_likelihood[:10]

[('my', 'hair'),
 ('this', 'product'),
 ('I', 'love'),
 ('my', 'skin'),
 ('a', 'little'),
 ('my', 'face'),
 ('a', 'lot'),
 ('love', 'this'),
 ('a', 'few'),
 ('a', 'bit')]

### raw freq

In [151]:
best_raw_freq = []
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(texts_all.split())
finder.apply_freq_filter(5)
best_bgrams = finder.nbest(bigram_measures.raw_freq, 5000)
for bgram in best_bgrams:
  if bgram in ngrams_all:
    best_raw_freq.append(bgram)

best_raw_freq[:10]

[('my', 'hair'),
 ('I', 'love'),
 ('this', 'product'),
 ('my', 'skin'),
 ('a', 'little'),
 ('this', 'is'),
 ('love', 'this'),
 ('my', 'face'),
 ('using', 'this'),
 ('the', 'product')]

Кажется, лучше всех справляется PMI

### Сгруппируем полученные коллокации по NE

In [221]:
products = ['lotion', 'brush', 'polish', 'oil', 'dryer']

In [223]:
for product in products:
  print(product)
  print('---')
  for col in best_pmi:
    if product in col:
      print(' '.join(col))
  print('\n')

lotion
---
drying lotion
facial lotion
best lotion
The lotion


brush
---
round brush
brush heads
brush head
shaving brush
The brush


polish
---
nail polish


oil
---
coconut oil
pre-shave oil
Moroccan oil
oil based
an oil


dryer
---
blow dryer
hair dryer
dryer I've




### Cпособ объединить синонимичные упоминания

* Кажется, проблему объединения "watch" и "smartwatch" можно решить даже регулярками. А можно, например, посмотреть на близость их эмбеддингов.
* Проблему с watch и Samsung Galaxy Watch можно решить с помощью синтаксических зависимостей: так будет выглядеть разметка UDPipe для предложения "I've bought myself Samsung Galaxy Watch":

In [225]:
print("""# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = english-gum-ud-2.6-200830
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = I've bought myself Samsung Galaxy Watch
1	I	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	3	nsubj	_	SpaceAfter=No|TokenRange=0:1
2	've	have	AUX	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	3	aux	_	TokenRange=1:4
3	bought	buy	VERB	VBN	Tense=Past|VerbForm=Part	0	root	_	TokenRange=5:11
4	myself	myself	PRON	PRP	Case=Acc|Number=Sing|Person=1|PronType=Prs|Reflex=Yes	3	iobj	_	TokenRange=12:18
5	Samsung	Samsung	PROPN	NNP	Number=Sing	7	compound	_	TokenRange=19:26
6	Galaxy	Galaxy	PROPN	NNP	Number=Sing	7	compound	_	TokenRange=27:33
7	Watch	Watch	PROPN	NNP	Number=Sing	3	obj	_	SpaceAfter=No|TokenRange=34:39
""")

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = english-gum-ud-2.6-200830
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = I've bought myself Samsung Galaxy Watch
1	I	I	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	3	nsubj	_	SpaceAfter=No|TokenRange=0:1
2	've	have	AUX	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	3	aux	_	TokenRange=1:4
3	bought	buy	VERB	VBN	Tense=Past|VerbForm=Part	0	root	_	TokenRange=5:11
4	myself	myself	PRON	PRP	Case=Acc|Number=Sing|Person=1|PronType=Prs|Reflex=Yes	3	iobj	_	TokenRange=12:18
5	Samsung	Samsung	PROPN	NNP	Number=Sing	7	compound	_	TokenRange=19:26
6	Galaxy	Galaxy	PROPN	NNP	Number=Sing	7	compound	_	TokenRange=27:33
7	Watch	Watch	PROPN	NNP	Number=Sing	3	obj	_	SpaceAfter=No|TokenRange=34:39



Здесь мы видим, что слова samsung и galaxy зависят от их вершины watch и являются кампаундом. То есть мы могли бы спокойно написать правило, извлекающее такие и другие похожие конструкции для товаров